In [ ]:
import pandas as pd
import re
from io import StringIO
from dateutil import parser as date_parser
import uuid

In [ ]:
FILE_PATH = '/content/translated_Maayan.txt'

USER_MAP = {
    'Maayan Sade': 'u_maayan',
    'Barbara': 'u_barbara',
}

In [ ]:
def parse_chat_log(file_path):
    """
    Reads a chat log file and robustly parses each message line into structured data.
    """
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            chat_content = f.read()
    except FileNotFoundError:
        print(f"Error: The file '{file_path}' was not found. Please ensure the file is present.")
        return pd.DataFrame()

    pattern = re.compile(
        r'^(?:.*?)\s*\[(\d{1,2}/\d{1,2}/\d{4}),\s*(\d{1,2}:\d{2}:\d{2})\s*(?:[AP]M)?\]\s*(.+?):\s*(.*)',
        re.IGNORECASE | re.MULTILINE
    )

    data = []

    for line in StringIO(chat_content):
        line = line.strip()
        if not line:
            continue

        match = pattern.match(line)

        if match:
            date_str, time_str, speaker, message = match.groups()

            clean_speaker = re.sub(r'\[.*?\]|\(|\)|\u200e|\u200f', '', speaker).strip()

            data.append({
                'Date': date_str,
                'Time': time_str,
                'Speaker': clean_speaker,
                'Message': message.strip()
            })
        elif data:
            data[-1]['Message'] += ' ' + line
    return pd.DataFrame(data)

def transform_to_sql_schema(df):
    """
    Transforms the raw DataFrame into the required 'messages' table schema,
    including the generated 'conv_id'.
    """
    if df.empty:
        return pd.DataFrame()

    unique_speakers = df['Speaker'].unique()
    df['sender_user_id'] = df['Speaker'].map(USER_MAP)
    receiver_id_map = {
        USER_MAP['Maayan Sade']: USER_MAP['Barbara'],
        USER_MAP['Barbara']: USER_MAP['Maayan Sade']
    }
    df['receiver_user_id'] = df['sender_user_id'].map(receiver_id_map)

    def generate_id_and_timestamp(row):
        msg_id = str(uuid.uuid4())
        try:
            dt_obj = date_parser.parse(f"{row['Date']} {row['Time']}", dayfirst=True)
            sent_at = dt_obj.strftime('%Y-%m-%d %H:%M:%S')
        except Exception:
            dt_obj = pd.NaT
            sent_at = None
        return pd.Series([msg_id, sent_at, dt_obj])

    df[['msg_id', 'sent_at','dt_object']] = df.apply(generate_id_and_timestamp, axis=1)
    user_ids = sorted(USER_MAP.values())
    df['conv_id'] = 'chat:' + '_'.join(user_ids)
    df = df.sort_values(by='dt_object').reset_index(drop=True)
    df = df.rename(columns={'Message': 'text'})
    df = df_messages.sort_values(['conv_id', 'sent_at']).reset_index(drop=True)

    block_id = (
        (df['conv_id'] != df['conv_id'].shift()) |
        (df['sender_user_id'] != df['sender_user_id'].shift())
    ).cumsum()

    df['block_id'] = block_id
    df_merged = (
        df.groupby('block_id', as_index=False)
          .agg({
              'msg_id': 'first',
              'conv_id': 'first',
              'sender_user_id': 'first',
              'receiver_user_id': 'first',
              'sent_at': 'min',
              'text': lambda x: "\n".join(x)
          })
    )
    df_merged = df_merged.drop(columns=['block_id'])
    df_merged = df_merged.sort_values(by='sent_at').reset_index(drop=True)
    df_merged['conv_turn'] = (df_merged['sender_user_id'] == df_merged['receiver_user_id'].shift()).cumsum() +1

    final_columns = [
        'msg_id',
        'conv_id',
        'conv_turn',
        'sender_user_id',
        'receiver_user_id',
        'sent_at',
        'text'
    ]

    final_df = df_merged[final_columns]
    return final_df

raw_df = parse_chat_log(FILE_PATH)
df_messages = transform_to_sql_schema(raw_df)

In [ ]:
df_messages

,msg_id,conv_id,conv_turn,sender_user_id,receiver_user_id,sent_at,text
0,e7c8b8a9-0b13-4bb0-aa1f-f698ada74644,chat:u_barbara_u_maayan,1,u_maayan,u_barbara,2025-01-13 12:49:06,Messages and calls are end-to-end encrypted. O...
1,dc1468e8-38d7-4e94-b1d1-17026e9b283b,chat:u_barbara_u_maayan,2,u_barbara,u_maayan,2025-01-13 12:49:06,"Hi, sweetheart.\nTell me, do you happen to hav..."
2,d1089e9b-fe4a-4fe1-97dd-53e7bf7cfbeb,chat:u_barbara_u_maayan,3,u_maayan,u_barbara,2025-01-13 21:25:20,I broke up with my girlfriend today
3,383a5d89-0276-4c7f-9cab-454a131b2473,chat:u_barbara_u_maayan,4,u_barbara,u_maayan,2025-01-13 21:25:20,"By any chance, could you replace me face-to-fa..."
4,801b2a09-47c7-4e51-b3bb-30f8a7976ddc,chat:u_barbara_u_maayan,5,u_maayan,u_barbara,2025-01-13 21:25:23,That's the excuse\nUmmmmm\nDoubt it
...,...,...,...,...,...,...,...
506,c1f4ac7f-061a-409a-b4bd-6d2d1d0e1080,chat:u_barbara_u_maayan,507,u_maayan,u_barbara,2025-11-04 16:49:59,My pleasure\nYou can also check mine Exam is 7...
507,7e631dc2-51f5-44d1-8785-2414469f1232,chat:u_barbara_u_maayan,508,u_barbara,u_maayan,2025-11-04 16:59:48,audio omitted
508,f904c237-3e96-40c1-8da4-51bf3a8c873e,chat:u_barbara_u_maayan,509,u_maayan,u_barbara,2025-11-04 17:02:02,I'm in a lecture\nAfter\n❤️
509,4fb2aa62-29f7-4089-afa9-0ce863078a9a,chat:u_barbara_u_maayan,510,u_barbara,u_maayan,2025-11-04 18:02:12,I wrote that I trust you with my eyes closed
